In [2]:
import numpy as np
import pandas as pd

Loading data

In [3]:
df = pd.read_csv('final file hopefully no errors.csv')
df['Len'] = [len(s) for s in df['Seq'].to_list()]

In [4]:
dfp = df.loc[df['Gen'] == 'ssRNA(+)']
dfn = df.loc[df['Gen'] == 'ssRNA(-)']

In [5]:
wdfp = dfp.sample(len(dfn))

In [108]:
wdfp = wdfp.reset_index(drop = True)
dfp = dfp.reset_index(drop = True)
dfn = dfn.reset_index(drop = True)

In [7]:
#load representative sequences into list
#take 10 shortest sequences from each df. 20 total repseqs.

repseqs = wdfp.sort_values('Len')[:10]['Seq'].to_list()

repseqs = repseqs + (dfn.sort_values('Len')[:10]['Seq'].to_list())

In [98]:
#taking all ATGC permutations for windows of length 1 to 6

import itertools

winlist16 = []

for i in range(1,7):

    for i in (itertools.product('ATGC', repeat=i)):
        winlist16.append(''.join(i))

In [ ]:
#load databases as dataframes

#set fragment/window size

window_sizes = list(range(5,7))

#set count threshold
###

#set other parameters
match_award      = 2
mismatch_penalty = -1
gap_penalty      = -1 # both for opening and extanding

#complete workflow

for win in window_sizes:
    winlist = [w for w in winlist16 if len(w) == win]
    winlist = winlist[388:]
    
    #taking all unique elements into winlist
    
#     for repseq in repseqs:
#         for i in range(0, len(repseq) - win + 1):
#             piece = repseq[i:i + win]
#             if piece not in winlist:
#                 winlist.append(piece)
                
    print('Done making %d windows' % len(winlist)) 
    
    #two dataframes to stores results
    
    resultp = pd.DataFrame(index = list(range(len(dfp))), columns = winlist)
    resultn = pd.DataFrame(index = list(range(len(dfn))), columns = winlist)
    
    c = 1
    
    for element in winlist:
        for i, r in dfp.iterrows():
            score = calculate_element_score3(match_award, mismatch_penalty, gap_penalty, element, r['Seq'])
            
            resultp.loc[i, element] = score
            
        for i, r in dfn.iterrows():
            score = calculate_element_score3(match_award, mismatch_penalty, gap_penalty, element, r['Seq'])
            
            resultn.loc[i, element] = score
            
        print('%d iterations done in winlist. %d remaining' % (c, (len(winlist) - c) ))
        
        c+=1

    resultp.to_csv('resultp window %d.csv' % win, index = False)
    resultn.to_csv('resultn window %d.csv' % win, index = False)

Done making 636 windows
1 iterations done in winlist. 635 remaining
2 iterations done in winlist. 634 remaining
3 iterations done in winlist. 633 remaining
4 iterations done in winlist. 632 remaining
5 iterations done in winlist. 631 remaining
6 iterations done in winlist. 630 remaining
7 iterations done in winlist. 629 remaining
8 iterations done in winlist. 628 remaining
9 iterations done in winlist. 627 remaining
10 iterations done in winlist. 626 remaining
11 iterations done in winlist. 625 remaining


In [112]:
resultp.to_csv('resultp window %d upto 388.csv' % win, index = False)
resultn.to_csv('resultn window %d upto 388.csv' % win, index = False)

In [55]:
window_sizes = list(range(20,21))

for win in window_sizes:
    winlist = []
    
    #taking all unique elements into winlist
    
    for repseq in repseqs:
        for i in range(0, len(repseq) - win + 1):
            piece = repseq[i:i + win]
            if piece not in winlist:
                winlist.append(piece)
                
    print('Done making %d windows' % len(winlist)) 

Done making 67742 windows


In [58]:
%%time
calculate_element_score2(match_award, mismatch_penalty, gap_penalty, winlist[0], seq2)

Wall time: 3.49 s


0.0

In [64]:
%%time
calculate_element_score4(match_award, mismatch_penalty, gap_penalty, winlist[0], seq2)

Wall time: 2.35 s


(0.0, 25, 0.625)

In [63]:
def calculate_element_score4(match_award, mismatch_penalty, gap_penalty, seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences

    # Generate DP table and traceback path pointer matrix
    score = np.zeros((m+1, n+1)).astype(int)      # the DP table
                                                    # to store the traceback path

    max_score = 0        # initial maximum score in DP table
    # Calculate DP table and mark pointers

    scores_list = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];

    perfect_score = min(len(seq1), len(seq2)) * match_award

    perfect_matches = np.count_nonzero(score == perfect_score)

    first_tolerance_match = np.count_nonzero(score == perfect_score - 1)
    second_tolerance_match = np.count_nonzero(score == perfect_score - 2)
    third_tolerance_match = np.count_nonzero(score == perfect_score - 3)

    final_score = perfect_matches + first_tolerance_match/(perfect_score * 2) + second_tolerance_match/(perfect_score * 4) + third_tolerance_match/(perfect_score * 8)

    max_obtained = score.max()
    
    fraction_obtained = max_obtained/perfect_score

    return final_score, max_obtained, fraction_obtained

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty

In [10]:
def calculate_element_score3(match_award, mismatch_penalty, gap_penalty, seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences

    # Generate DP table and traceback path pointer matrix
    score = np.zeros((m+1, n+1)).astype(int)      # the DP table
                                                    # to store the traceback path

    max_score = 0        # initial maximum score in DP table
    # Calculate DP table and mark pointers

    scores_list = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];

    perfect_score = min(len(seq1), len(seq2)) * match_award

    perfect_matches = np.count_nonzero(score == perfect_score)

    first_tolerance_match = np.count_nonzero(score == perfect_score - 1)
    second_tolerance_match = np.count_nonzero(score == perfect_score - 2)
    third_tolerance_match = np.count_nonzero(score == perfect_score - 3)

    final_score = perfect_matches + first_tolerance_match/(perfect_score * 2) + second_tolerance_match/(perfect_score * 4) + third_tolerance_match/(perfect_score * 8)
    
    return final_score

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty

In [8]:
def calculate_element_score2(match_award, mismatch_penalty, gap_penalty, seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences

    # Generate DP table and traceback path pointer matrix
    score = np.zeros((m+1, n+1)).astype(int)      # the DP table
    pointer = np.zeros((m+1, n+1)).astype(int)    # to store the traceback path

    max_score = 0        # initial maximum score in DP table
    # Calculate DP table and mark pointers

    scores_list = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] == 0:
                pointer[i][j] = 0 # 0 means end of the path
            if score[i][j] == score_left:
                pointer[i][j] = 1 # 1 means trace up
            if score[i][j] == score_up:
                pointer[i][j] = 2 # 2 means trace left
            if score[i][j] == score_diagonal:
                pointer[i][j] = 3 # 3 means trace diagonal
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];

    perfect_score = min(len(seq1), len(seq2)) * match_award

    perfect_matches = np.count_nonzero(score == perfect_score)

    first_tolerance_match = np.count_nonzero(score == perfect_score - 1)
    second_tolerance_match = np.count_nonzero(score == perfect_score - 2)
    third_tolerance_match = np.count_nonzero(score == perfect_score - 3)

    final_score = perfect_matches + first_tolerance_match/(perfect_score * 2) + second_tolerance_match/(perfect_score * 4) + third_tolerance_match/(perfect_score * 8)
    
    return final_score

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty

In [14]:
def calculate_element_score(match_award, mismatch_penalty, gap_penalty, seq1, seq2):

    m, n = len(seq1), len(seq2)  # length of two sequences

    # Generate DP table and traceback path pointer matrix
    score = np.zeros((m+1, n+1)).astype(int)      # the DP table
    pointer = np.zeros((m+1, n+1)).astype(int)    # to store the traceback path

    max_score = 0        # initial maximum score in DP table
    # Calculate DP table and mark pointers

    scores_list = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] == 0:
                pointer[i][j] = 0 # 0 means end of the path
            if score[i][j] == score_left:
                pointer[i][j] = 1 # 1 means trace up
            if score[i][j] == score_up:
                pointer[i][j] = 2 # 2 means trace left
            if score[i][j] == score_diagonal:
                pointer[i][j] = 3 # 3 means trace diagonal
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];

    align1, align2 = '', ''    # initial sequences

    i,j = max_i,max_j    # indices of path starting point

    #traceback, follow pointers
    while pointer[i][j] != 0:
        if pointer[i][j] == 3:
            align1 += seq1[i-1]
            align2 += seq2[j-1]
            i -= 1
            j -= 1
        elif pointer[i][j] == 2:
            align1 += '-'
            align2 += seq2[j-1]
            j -= 1
        elif pointer[i][j] == 1:
            align1 += seq1[i-1]
            align2 += '-'
            i -= 1

    identity, align, symbol = finalize(align1, align2, scores_list)

    perfect_score = min(len(seq1), len(seq2)) * match_award

    perfect_matches = np.count_nonzero(score == perfect_score)

    first_tolerance_match = np.count_nonzero(score == perfect_score - 1)
    second_tolerance_match = np.count_nonzero(score == perfect_score - 2)
    third_tolerance_match = np.count_nonzero(score == perfect_score - 3)

    final_score = perfect_matches + first_tolerance_match/(perfect_score * 2) + second_tolerance_match/(perfect_score * 4) + third_tolerance_match/(perfect_score * 8)
    
    return final_score

fucntions

In [15]:
def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty

In [16]:
def finalize(align1, align2, scores_list):
    align1 = align1[::-1]    #reverse sequence 1
    align2 = align2[::-1]    #reverse sequence 2
    
    i,j = 0,0
    
    #calcuate identity, score and aligned sequeces
    symbol = ''
    found = 0
    score = 0
    identity = 0
    for i in range(0,len(align1)):
        # if two AAs are the same, then output the letter
        if align1[i] == align2[i]:                
            symbol = symbol + align1[i]
            identity = identity + 1
            score += match_score(align1[i], align2[i])
    
        # if they are not identical and none of them is gap
        elif align1[i] != align2[i] and align1[i] != '-' and align2[i] != '-': 
            score += match_score(align1[i], align2[i])
            symbol += ' '
            found = 0
    
        #if one of them is a gap, output a space
        elif align1[i] == '-' or align2[i] == '-':          
            symbol += ' '
            score += gap_penalty
    
    
    #identity2 = float(identity) / len(align1) * 100
    
    #print ('Identity =', "%3.3f" % identity2, 'percent')
    #print ('Score =', score)
    #print (align1)
    #print (symbol)
    #print (align2)
    #print(n)
    
    return identity, align1, symbol

In [35]:
seq1     = 'ATGCATGCCGCGAGTC'
seq2     = 'TTACATGCATGCCGCGAGTCAGGATGTGGATAGAAGCCCCCCCCACAGGATGTACAGGATGTGGATAGAAGCCCCCCCCCCCTGGATAGAAGGATAGGGGGTACAGGATGTGGATAGAAGGGGGGGGTAAGGATGTGGATAGAAGGGGGGGGAGATACAGGATGTGGGGGGGGGGCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGGGGGGCTGAAGATTTTTTTTTTTTTTTTTTTTTACAGATACAGGATGTGCTGTTTTTTTTTTTTTTTTTTTTAATACATACACACATACAGGTGTGGTACAGATACAACAGATACAGGTTAGCGATACAGATACAGATACAGGATGTGCTGAATACATACACACATACAGGTGTGGTACAGATACAACAGATACAGGTTAGCGATACAGATACAGATACAGGATGTGCTGAATACATACACACATACAGGTGTGGTACAGATACAACAGATACAGGTTAGCGATACAGATACAGATACAGGATGTGCTGAATACATACACACATACAGGTGTGGTACAGATCCCCCCCCCCCCCCCCCCCCCCCCCCCCCACAACAGATACAGGTTAGCGATACATACTTTTTTTTTTCCCCCCCCCCCCCCCCCCCCCCCCCCCCCTTTTTTTTTTATACACACATACAGGTGTGGTACAGATACAACAGATACAGGTTAGCGATACA'

water(seq1, seq2)

Identity = 100.000 percent
Score = 32
ATGCATGCCGCGAGTC
ATGCATGCCGCGAGTC
ATGCATGCCGCGAGTC


Original

In [18]:
#This software is a free software. Thus, it is licensed under GNU General Public License.
#Python implementation to Smith-Waterman Algorithm for Homework 1 of Bioinformatics class.
#Forrest Bao, Sept. 26 <http://fsbao.net> <forrest.bao aT gmail.com>

# zeros() was origianlly from NumPy.
# This version is implemented by alevchuk 2011-04-10
def zeros(shape):
    retval = []
    for x in range(shape[0]):
        retval.append([])
        for y in range(shape[1]):
            retval[-1].append(0)
    return retval

match_award      = 2
mismatch_penalty = -1
gap_penalty      = -1 # both for opening and extanding

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty

def finalize(align1, align2, scores_list):
    align1 = align1[::-1]    #reverse sequence 1
    align2 = align2[::-1]    #reverse sequence 2
    
    i,j = 0,0
    
    #calcuate identity, score and aligned sequeces
    symbol = ''
    found = 0
    score = 0
    identity = 0
    for i in range(0,len(align1)):
        # if two AAs are the same, then output the letter
        if align1[i] == align2[i]:                
            symbol = symbol + align1[i]
            identity = identity + 1
            score += match_score(align1[i], align2[i])
    
        # if they are not identical and none of them is gap
        elif align1[i] != align2[i] and align1[i] != '-' and align2[i] != '-': 
            score += match_score(align1[i], align2[i])
            symbol += ' '
            found = 0
    
        #if one of them is a gap, output a space
        elif align1[i] == '-' or align2[i] == '-':          
            symbol += ' '
            score += gap_penalty
    
    
    identity = float(identity) / len(align1) * 100
    
    print ('Identity =', "%3.3f" % identity, 'percent')
    print ('Score =', score)
    print (align1)
    print (symbol)
    print (align2)
    #print(n)

def water(seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences
    
    # Generate DP table and traceback path pointer matrix
    score = zeros((m+1, n+1))      # the DP table
    pointer = zeros((m+1, n+1))    # to store the traceback path
    
    max_score = 0        # initial maximum score in DP table
    # Calculate DP table and mark pointers
    
    scores_list = []
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] == 0:
                pointer[i][j] = 0 # 0 means end of the path
            if score[i][j] == score_left:
                pointer[i][j] = 1 # 1 means trace up
            if score[i][j] == score_up:
                pointer[i][j] = 2 # 2 means trace left
            if score[i][j] == score_diagonal:
                pointer[i][j] = 3 # 3 means trace diagonal
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];
            
            #scores_list.append((score,i,j))
    
    
    align1, align2 = '', ''    # initial sequences
    
    i,j = max_i,max_j    # indices of path starting point
    
    #traceback, follow pointers
    while pointer[i][j] != 0:
        if pointer[i][j] == 3:
            align1 += seq1[i-1]
            align2 += seq2[j-1]
            i -= 1
            j -= 1
        elif pointer[i][j] == 2:
            align1 += '-'
            align2 += seq2[j-1]
            j -= 1
        elif pointer[i][j] == 1:
            align1 += seq1[i-1]
            align2 += '-'
            i -= 1

    finalize(align1, align2, scores_list)

In [ ]:
def needle(seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences
    
    # Generate DP table and traceback path pointer matrix
    score = zeros((m+1, n+1))      # the DP table
   
    # Calculate DP table
    for i in range(0, m + 1):
        score[i][0] = gap_penalty * i
    for j in range(0, n + 1):
        score[0][j] = gap_penalty * j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score[i - 1][j - 1] + match_score(seq1[i-1], seq2[j-1])
            delete = score[i - 1][j] + gap_penalty
            insert = score[i][j - 1] + gap_penalty
            score[i][j] = max(match, delete, insert)

    # Traceback and compute the alignment 
    align1, align2 = '', ''
    i,j = m,n # start from the bottom right cell
    while i > 0 and j > 0: # end toching the top or the left edge
        score_current = score[i][j]
        score_diagonal = score[i-1][j-1]
        score_up = score[i][j-1]
        score_left = score[i-1][j]

        if score_current == score_diagonal + match_score(seq1[i-1], seq2[j-1]):
            align1 += seq1[i-1]
            align2 += seq2[j-1]
            i -= 1
            j -= 1
        elif score_current == score_left + gap_penalty:
            align1 += seq1[i-1]
            align2 += '-'
            i -= 1
        elif score_current == score_up + gap_penalty:
            align1 += '-'
            align2 += seq2[j-1]
            j -= 1

    # Finish tracing up to the top left cell
    while i > 0:
        align1 += seq1[i-1]
        align2 += '-'
        i -= 1
    while j > 0:
        align1 += '-'
        align2 += seq2[j-1]
        j -= 1

    finalize(align1, align2)


recycled code

In [ ]:
perfect_hits = 0
tolerated_hits = np.zeros(tolerance)
lowest_tolerance = min(len(seq1), len(seq2)) - tolerance
score_to_compare = max_score
flattened_scores = score.flatten()
flattened_scores.sort()

if identity == min(len(seq1), len(seq2)):
    perfect_hits = np.count_nonzero(score == max_score)
    
    for i in range(lowest_tolerance, identity):
        tolerated_hits[identity - (i+1)] = np.count_nonzero(score == score_to_compare)
        score_to_compare = list(set(flattened_scores))[-(i - lowest_tolerance + 2 )]

else:
    for i in range(lowest_tolerance, identity + 1):
        tolerated_hits[identity - i] = np.count_nonzero(score == score_to_compare)
        score_to_compare = list(set(flattened_scores))[-(i - lowest_tolerance + 2 )]
        
print('purrfect %d' %perfect_hits)
for i in range(tolerance):
    print('tolerating %d mismatches/gaps: %d' %  (i+1, tolerated_hits[i]))